# (Title to be decided)

*This is the file for the code of our group assignment. Under development.*

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## 1. Read Data from External File

Read data in external files. For convenience, the original `.pcapng` file has been converted into `.json` and `.csv` with Wireshark.

In [2]:
# Read the JSON including all details of packets
#   errors="surrogateescape" to pass the encoding errors raised by contents in HTTP packets
with open("./data/traffic.json", "r", encoding="utf-8", errors="surrogateescape") as f:
    dataset_json=json.load(f)
    print(f"{len(dataset_json)} packets read")

# print one for checking
dataset_json[0]

10625 packets read


{'_index': 'packets-XXXX-XX-XX',
 '_type': 'doc',
 '_score': None,
 '_source': {'layers': {'pkt_comment': {'frame.comment': '13682128230572000042,china',
    'frame.comment_tree': {'_ws.expert': {'frame.comment.expert': '',
      '_ws.expert.message': '13682128230572000042,china',
      '_ws.expert.severity': '1048576',
      '_ws.expert.group': '184549376'}}},
   'frame': {'frame.interface_id': '0',
    'frame.interface_id_tree': {'frame.interface_name': 'unknown'},
    'frame.encap_type': '7',
    'frame.time': 'Not representable',
    'frame.offset_shift': '0.000000000',
    'frame.time_epoch': '-1460980371.753735000',
    'frame.time_delta': '0.000000000',
    'frame.time_delta_displayed': '0.000000000',
    'frame.time_relative': '0.000000000',
    'frame.number': '1',
    'frame.len': '58',
    'frame.cap_len': '58',
    'frame.marked': '0',
    'frame.ignored': '0',
    'frame.protocols': 'raw:ip:udp:dns',
    'frame.coloring_rule.name': 'UDP',
    'frame.coloring_rule.string': 

In [3]:
# # TODO: UNUSED - REMOVE LATER
# # Read the CSV
# # TODO: Info in CSV should be covered by the one in JSON.
# #       Remove it in the future if not used
# dataset_csv=pd.read_csv("./data/traffic.csv")

# dataset_csv

## 2. Data Preprocessing

### 2.1 Extract dataset labels

In [4]:
# Extract labels(y) from JSON
dataset_y = np.array(list(map(
    lambda packet_json: (packet_json["_source"]["layers"]["pkt_comment"]["frame.comment"]).split(",")[1], 
    dataset_json
)))

print(f"Labels include {np.unique(dataset_y)}")
print(dataset_y.shape)

Labels include ['china' 'india' 'us']
(10625,)


### 2.2 Extract features

In [5]:
# Extract source IP, source IP host, destination IP and destination IP host from dataset
dataset_X_ip_src = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.src"], 
    dataset_json
)))
dataset_X_ip_src_host = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.src_host"], 
    dataset_json
)))
dataset_X_ip_dst = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.dst"], 
    dataset_json
)))
dataset_X_ip_dst_host = np.array(list(map(
    lambda packet_json: packet_json["_source"]["layers"]["ip"]["ip.dst_host"], 
    dataset_json
)))

# print first ones for checking
print(dataset_X_ip_src[0])
print(dataset_X_ip_src_host[0])
print(dataset_X_ip_dst[0])
print(dataset_X_ip_dst_host[0])


10.11.1.3
10.11.1.3
8.8.8.8
dns.google


### 2.3 Check data sanity and deal with missing fields

In [6]:
# Data sanity check
pattern_not_ipv4 = re.compile(r"(?!((\b25[0-5]|\b2[0-4][0-9]|\b[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}))")
pattern_not_ipv4_nor_domain = re.compile(r"(?!(((\b25[0-5]|\b2[0-4][0-9]|\b[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}))|[-_0-9a-zA-Z]+\.[-_0-9a-zA-Z]+)")

print("Source/dest IP not an IPv4 address:")
print(list(filter(pattern_not_ipv4.match, dataset_X_ip_src)))
print(list(filter(pattern_not_ipv4.match, dataset_X_ip_dst)))
print("Source/dest host not an IPv4 nor a domain:")
print(list(filter(pattern_not_ipv4_nor_domain.match, dataset_X_ip_src_host)))
print(list(filter(pattern_not_ipv4_nor_domain.match, dataset_X_ip_dst_host)))

print()

# Fill the missing `dst.host`
#
# Find "no-data" rows
missing_dst_host_indexes = np.where(dataset_X_ip_dst_host=='no-data')[0]
print(f"Indexes of rows with missing 'dst.host': {missing_dst_host_indexes}")

# Fill the field with the IP address in `dst`, following the way of other rows
for i in missing_dst_host_indexes:
    dataset_X_ip_dst_host[i] = dataset_X_ip_dst[i]

print("Missing 'dst.host' filled")

Source/dest IP not an IPv4 address:
[]
[]
Source/dest host not an IPv4 nor a domain:
[]
['no-data', 'no-data']

Indexes of rows with missing 'dst.host': [960 964]
Missing 'dst.host' filled


### 2.4 Bi-gram for similarity between IPs and domains

In [7]:
# bi-gram for similar IP and hostnames
bigram_vec = CountVectorizer(ngram_range=(2, 2), token_pattern=r"(?u)[^.]+")
dataset_X_ip_src_bg = bigram_vec.fit_transform(dataset_X_ip_src).toarray()
dataset_X_ip_src_host_bg = bigram_vec.fit_transform(dataset_X_ip_src_host).toarray()
dataset_X_ip_dst_bg = bigram_vec.fit_transform(dataset_X_ip_dst).toarray()
dataset_X_ip_dst_host_bg = bigram_vec.fit_transform(dataset_X_ip_dst_host).toarray()

print(dataset_X_ip_src_bg.shape)
print(dataset_X_ip_src_host_bg.shape)
print(dataset_X_ip_dst_bg.shape)
print(dataset_X_ip_dst_host_bg.shape)

dataset_X = np.concatenate((
    dataset_X_ip_src_bg, 
    dataset_X_ip_src_host_bg,
    dataset_X_ip_dst_bg, 
    dataset_X_ip_dst_host_bg), axis=1)

print("")
print(dataset_X.shape)


(10625, 1402)
(10625, 867)
(10625, 1629)
(10625, 1059)

(10625, 4957)


### 2.5 Normalization

In [8]:
# Normalization
dataset_X = MinMaxScaler().fit_transform(dataset_X)
print(dataset_X.shape)

(10625, 4957)


### 2.6 Principal Component Analysis (PCA)

In [9]:
# # PCA - check the 95% with figure
# pca = PCA()
# pca.fit(dataset_X)
# pca_cumsum = np.cumsum(pca.explained_variance_ratio_)

# plt.figure()
# plt.axis([0, dataset_X.shape[1], 0, 1]) 
# plt.plot(pca_cumsum)
# plt.axhline(y=0.95,color='red')
# plt.xlabel("dimensions")
# plt.ylabel("explained variance")
# plt.grid(True)
# plt.show()

In [10]:
# Apply PCA for compression
pca = PCA(n_components=0.95)
dataset_X = pca.fit_transform(dataset_X)

print(dataset_X.shape)

(10625, 650)


### 2.7 Dataset split

In [11]:
# Split the dataset into training set and testing set
train_X, test_X, train_y, test_y = train_test_split(dataset_X, dataset_y, random_state=12345)

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(7968, 650)
(7968,)
(2657, 650)
(2657,)


### 2.X kNN classification as an example for using the processed dataset

(And for checking the preprocess performance at the early stage)

TODO: REMOVE BEFORE FINAL SUBMIT

In [12]:
# kNN classification test
#
# TODO: A brief test for evaluating performance during writing preprocessin code, 
#       gonna be removed later
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)
print(f"Accuracy on test set: {knn.score(test_X, test_y):.4f}")

Accuracy on test set: 0.9398


/Users/yqmtf/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:189: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


## 3. Model Training

### 3.1 Logistic Regression

TODO

In [18]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
LR = LogisticRegression()
print(train_X.shape)
print(train_y.shape)


#predict_results=LR.predict(test_X)
#print(accuracy_score(predict_results, test_y))


(7968, 650)
(7968,)


### 3.2 Support Vector Machine (SVM)

Implementing a kernelized SVM

In [52]:
#For the linear kernel (kernel_type='linear') svm
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
SVM_linear = SVC(kernel='linear') 
SVM_linear.fit(train_X, train_y)
predict_results=SVM_linear.predict(test_X)
print(f"Accuracy on test set: {accuracy_score(predict_results, test_y):.4f}")

Accuracy on test set: 0.8965


In [32]:
#For the radial basis function kernel (kernel_type='rbf') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_rbf = SVC(kernel='rbf')  
SVM_rbf.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_rbf.score(test_X, test_y):.4f}")

Accuracy on test set: 0.8991


In [33]:
#For the polynomial kernel (kernel_type='poly') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_poly = SVC(kernel='poly') 
SVM_poly.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_poly.score(test_X, test_y):.4f}")

Accuracy on test set: 0.8995


In [49]:
#For the polynomial kernel (kernel_type='poly') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_poly = SVC(C=1, kernel='poly', gamma=20, decision_function_shape='ovr')
SVM_poly.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_poly.score(test_X, test_y):.4f}")

Accuracy on test set: 0.9037


In [48]:
#For the radial basis function kernel (kernel_type='rbf') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_rbf_gamma = SVC(C=1, kernel='rbf', gamma=10, decision_function_shape='ovr')
SVM_rbf_gamma.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_rbf_gamma.score(test_X, test_y):.4f}")

Accuracy on test set: 0.8946


In [36]:
#For the radial basis function kernel (kernel_type='rbf') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_rbf_modify = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto',
                 coef0=0.0, shrinking=True, probability=False,
                 tol=0.001, cache_size=200, class_weight=None,
                 verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)
SVM_rbf_modify.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_rbf_modify.score(test_X, test_y):.4f}")

Accuracy on test set: 0.7347


In [37]:
#For the sigmoid function kernel (kernel_type='sigmoid') svm
from sklearn.svm import SVC  # Support Vector Classifier
SVM_sig = SVC(C=1, kernel='sigmoid', gamma=10, decision_function_shape='ovr')
SVM_sig.fit(train_X, train_y)
print(f"Accuracy on test set: {SVM_sig.score(test_X, test_y):.4f}")

Accuracy on test set: 0.3801


### 3.3 (?) neural network

Maybe?

## 4. Model Testing and Comparison

TODO